In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV


df = pd.read_excel("Scoring_9.xlsx")
df.head()

,startdate,totalamount,monthly_payment,paymentperiod,collateral_type,flag_sozaem,ip_sign,sum_monthlypayment,sum_outstandingamount,max_dpd_before_startdate,...,incomes,incomes_bdki,work_experience,incomes_bdki_int,incomes_bdki_int_indxd,age,dir,credit_history_length,term,GB_flag
0,2021-04-29,19200000.0,150761.71,578,604,0,0,194760.9478,558508.0,168.0,...,233333.3333,100 000 - 150 000 тенге в месяц,NaN,150000.0,165000.0,50.11,0.83,91,4658,0
1,2021-06-14,5600000.0,76349.66,578,604,0,0,0.0000,0.0,2.0,...,524960.0000,150 000 - 200 000 тенге в месяц,39.0,200000.0,220000.0,34.19,0.00,132,3652,0
2,2020-02-20,11200000.0,110187.75,578,604,0,0,175000.0000,2100000.0,NaN,...,884964.8000,NaN,67.0,NaN,NaN,33.91,0.20,112,5479,0
3,2020-07-13,16000000.0,0.00,578,604,0,0,198537.6190,92204.0,NaN,...,241514.2857,NaN,NaN,NaN,NaN,28.49,0.82,81,5478,0
4,2021-05-19,9500000.0,167892.00,578,604,0,0,0.0000,0.0,2624.0,...,271900.0000,NaN,27.0,NaN,NaN,36.11,0.00,180,3652,0


In [2]:
# Data preprocessing
median_columns = [
    'gender', 'dependants', 
    'maritalstatus', 
    'incomes', 'work_experience',
    'incomes_bdki_int', 
    'incomes_bdki_int_indxd'
    ]
mean_columns = [
    'sum_outstandingamount',
    'max_dpd_before_startdate'
]    
df['startdate'] = pd.to_numeric(pd.DatetimeIndex(df['startdate']).year)
df[df['max_dpd_before_startdate'] == -9999.0] = 0
df = df.drop(['incomes_bdki'], axis=1, errors='ignore')

for i in median_columns:
  df[i] = df[i].fillna(df[i].median())

for i in mean_columns:
  df[i] = df[i].fillna(df[i].mean())

df.isna().sum()
df.head()

,startdate,totalamount,monthly_payment,paymentperiod,collateral_type,flag_sozaem,ip_sign,sum_monthlypayment,sum_outstandingamount,max_dpd_before_startdate,...,cnt_90d,incomes,work_experience,incomes_bdki_int,incomes_bdki_int_indxd,age,dir,credit_history_length,term,GB_flag
0,2021,19200000.0,150761.71,578,604,0,0,194760.9478,558508.0,168.000000,...,4,233333.3333,46.0,150000.0,165000.0,50.11,0.83,91,4658,0
1,2021,5600000.0,76349.66,578,604,0,0,0.0000,0.0,2.000000,...,3,524960.0000,39.0,200000.0,220000.0,34.19,0.00,132,3652,0
2,2020,11200000.0,110187.75,578,604,0,0,175000.0000,2100000.0,222.245094,...,3,884964.8000,67.0,200000.0,240000.0,33.91,0.20,112,5479,0
3,2020,16000000.0,0.00,578,604,0,0,198537.6190,92204.0,222.245094,...,1,241514.2857,46.0,200000.0,240000.0,28.49,0.82,81,5478,0
4,2021,9500000.0,167892.00,578,604,0,0,0.0000,0.0,2624.000000,...,4,271900.0000,27.0,200000.0,240000.0,36.11,0.00,180,3652,0


In [3]:
y = df.loc[:,'GB_flag']
X = df.drop(columns=['GB_flag'])

In [4]:
from sklearn.metrics import make_scorer, roc_auc_score
def gini_normalized(y_actual, y_pred):
    """Simple normalized Gini based on Scikit-Learn's roc_auc_score"""
    gini = lambda a, p: 2 * roc_auc_score(a, p) - 1
    return gini(y_actual, y_pred) / gini(y_actual, y_actual)

In [17]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import RepeatedKFold

model = ElasticNet(alpha=0.01, l1_ratio=0.4)

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

gini = cross_val_score(model, X, y, scoring=make_scorer(gini_normalized), cv=cv, n_jobs=-1)


gini = np.absolute(gini)
print('Gini score: %.3f ' % (np.mean(gini)))

/home/codespace/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.534e+02, tolerance: 5.318e-02
  model = cd_fast.enet_coordinate_descent(
/home/codespace/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.554e+02, tolerance: 5.347e-02
  model = cd_fast.enet_coordinate_descent(
/home/codespace/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duali

Gini score: 0.648 


/home/codespace/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.522e+02, tolerance: 5.299e-02
  model = cd_fast.enet_coordinate_descent(
